# 说说Python中的闭包 - Closure
https://betacat.online/posts/python-closure/

## 闭包的概念

在一些语言中，在函数中可以（嵌套）定义另一个函数时，如果内部的函数引用了外部的函数的变量，则可能产生闭包。闭包可以用来在一个函数与一组“私有”变量之间创建关联关系。在给定函数被多次调用的过程中，这些私有变量能够保持其持久性。

In other words, 就是当某个函数被当成对象返回时，夹带了外部变量，就形成了一个闭包。

In [4]:
def make_printer(msg):
    def printer():
        print(msg)  # 夹带私货（外部变量）
    return printer  # 返回的是函数，带私货的函数
printer = make_printer('Foo!')
printer()

Foo!


## 如何理解闭包
闭包存在有什么意义呢？为什么需要闭包？

我个人认为，闭包存在的意义就是它夹带了外部变量（私货），如果它不夹带私货，它和普通的函数就没有任何区别。

同一个的函数夹带了不同的私货，就实现了不同的功能。其实你也可以这么理解，闭包和面向接口编程的概念很像，可以把闭包理解成轻量级的接口封装。

- 接口定义了一套对方法签名的约束规则。

In [13]:
def tag(tag_name):
    def add_tag(content):
        return "<{0}>{1}</{0}>".format(tag_name, content)
    return add_tag

content = 'Hello'

add_tag = tag('a')
print(add_tag(content))  # <a>Hello</a>

add_tag = tag('b')
print(add_tag(content))  # <b>Hello</b>

<a>Hello</a>
<b>Hello</b>


在这个例子里，我们想要一个给content加tag的功能，但是具体的tag_name是什么样子的要根据实际需求来定，对外部调用的接口已经确定，就是add_tag(content)。如果按照面向接口方式实现，我们会先把add_tag写成接口，指定其参数和返回类型，然后分别去实现a和b的add_tag。

但是在闭包的概念中，add_tag就是一个函数，它需要tag_name和content两个参数，只不过tag_name这个参数是打包带走的。所以一开始时就可以告诉我怎么打包，然后带走就行。

## 何时使用闭包
其实闭包在Python中很常见，只不过你没特别注意这就是一个闭包。比如Python中的装饰器Decorator，假如你需要写一个带参数的装饰器，那么一般都会生成闭包。

为什么？因为Python的装饰器是一个固定的函数接口形式。它要求你的装饰器函数（或装饰器类）必须接受一个函数并返回一个函数：

那么如果你的装饰器如果带参数呢？那么你就需要在原来的装饰器上再包一层，用于接收这些参数。

这些参数（私货）传递到内层的装饰器里后，闭包就形成了。所以说当你的装饰器需要自定义参数时，一般都会形成闭包。（类装饰器例外）

In [22]:
def html_tags(tag_name):
    def wrapper_(func):
        def wrapper(*args, **kwargs):
            content = func(*args, **kwargs)
            return "<{tag}>{content}</{tag}>".format(tag=tag_name, content=content)
        return wrapper
    return wrapper_

@html_tags('b')
def hello(name='Toby'):
    return 'Hello {}!'.format(name)
# 不用@的写法如下
# hello = html_tag('b')(hello)
# html_tag('b') 是一个闭包，它接受一个函数，并返回一个函数


print(hello())  # <b>Hello Toby!</b>

print(hello('world'))  # <b>Hello world!</b>

<b>Hello Toby!</b>
<b>Hello world!</b>


其实闭包函数相对与普通函数会多出一个__closure__的属性，里面定义了一个元组用于存放所有的cell对象，每个cell对象一一保存了这个闭包中所有的外部变量。

## Something interesting: \__closure__ and cell_contents

In [38]:
hello1 = html_tags('html_tags')

#### 返回cell元组

In [39]:
hello1.__closure__

(<cell at 0x1046a97c8: str object at 0x1046dd7f0>,)

In [41]:
hello1('contents').__closure__

(<cell at 0x1046a9c78: str object at 0x1003b5530>,
 <cell at 0x1046a97c8: str object at 0x1046dd7f0>)

#### 外部变量

In [40]:
hello1.__closure__[0].cell_contents

'html_tags'

In [42]:
hello1('contents').__closure__[0].cell_contents

'contents'

In [43]:
hello1('contents').__closure__[1].cell_contents

'html_tags'